In [ ]:
from llamabot import SimpleBot


In [ ]:
bot = SimpleBot("You are Richard Feynman", model_name='codellama')
bot("how are you doing?")


In [ ]:
"""Dispatch"""
import contextvars

from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOllama, ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

from llamabot.config import default_language_model
from llamabot.recorder import autorecord

prompt_recorder_var = contextvars.ContextVar("prompt_recorder")


In [ ]:
model_name = "codellama"
temperature = 0.0
verbose = True
streaming = True

llm = ChatOpenAI(
    model_name=model_name,
    temperature=temperature,
    streaming=streaming,
    verbose=verbose,
    callback_manager=BaseCallbackManager(
        handlers=[StreamingStdOutCallbackHandler()] if streaming else []
    ),
)

llm()


In [ ]:
def dispatch_model(
    model_name,
    temperature=0.0,
    streaming=True,
    verbose=True,
):
    """Dispatch and create the right model"""
    pass


class SimpleBot:
    """Simple Bot that is primed with a system prompt, accepts a human message,
    and sends back a single response.

    This bot does not retain chat history.
    """

    def __init__(
        self,
        system_prompt,
        temperature=0.0,
        model_name="codellama",
        streaming=True,
        verbose=True,
    ):
        """Initialize the SimpleBot.

        :param system_prompt: The system prompt to use.
        :param temperature: The model temperature to use.
            See https://platform.openai.com/docs/api-reference/completions/create#completions/create-temperature
            for more information.
        :param model_name: The name of the OpenAI model to use.
        :param streaming: (LangChain config) Whether to stream the output to stdout.
        :param verbose: (LangChain config) Whether to print debug messages.
        """
        self.system_prompt = system_prompt
        self.model = ChatOllama(
            model_name=model_name,
            temperature=temperature,
            streaming=streaming,
            verbose=verbose,
            callback_manager=BaseCallbackManager(
                handlers=[StreamingStdOutCallbackHandler()] if streaming else []
            ),
        )
        self.chat_history = []
        self.model_name = model_name

    def __call__(self, human_message: str) -> AIMessage:
        """Call the SimpleBot.

        :param human_message: The human message to use.
        :return: The response to the human message, primed by the system prompt.
        """

        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=human_message),
        ]
        response = self.model(messages)
        self.chat_history.append(HumanMessage(content=human_message))
        self.chat_history.append(response)
        autorecord(human_message, response.content)
        return response


In [ ]:
bot = SimpleBot("You are a coding expert", model_name="codellama")


In [ ]:
bot("how do you write a FastAPI endpoint?")
